Reinforcement Learning with OpenAI Gym
---
This notebook will create and test different reinforcement learning agents and environments.

In [1]:
import tensorflow as tf
import gym
import os

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load the Environment
---
Call `gym.make("environment name")` to load a new environment.

Check out the list of available environments at <https://gym.openai.com/envs/>

Edit this cell to load different environments!

In [2]:
# TODO: Load an environment
env = gym.make("CartPole-v1")

[2018-08-03 10:33:18,575] Making new env: CartPole-v1


In [3]:
# TODO: Print observation and action spaces
print(env.observation_space)
print(env.action_space)

Box(4,)
Discrete(2)


Run an Agent
---

Reset the environment before each run with `env.reset`

Step forward through the environment to get new observations and rewards over time with `env.step`

`env.step` takes a parameter for the action to take on this step and returns the following:
- Observations for this step
- Rewards earned this step
- "Done", a boolean value indicating if the game is finished
- Info - some debug information that some environments provide. 

In [4]:
# TODO Make a random agent
games_to_play = 10
 
for i in range(games_to_play):
    # Reset the environment
    obs = env.reset()
    episode_rewards = 0
    done = False
     
    while not done:
        # Render the environment so we can watch
        env.render()
         
        # Choose a random action
        action = env.action_space.sample()
         
        # Take a step in the environment with the chosen action
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
 
    # Print episode total rewards when done
    print(episode_rewards)
     
# Close the environment
env.close()

13.0
15.0
17.0
12.0
17.0
21.0
15.0
13.0
13.0
11.0


Policy Gradients
---
The policy gradients algorithm records gameplay over a training period, then runs the results of the actions chosen through a neural network, making successful actions that resulted in a reward more likely, and unsuccessful actions less likely.

In [5]:
# TODO Build the policy gradient neural network
class Agent:
    def __init__(self, num_actions, state_size):
         
        initializer = tf.contrib.layers.xavier_initializer()
         
        self.input_layer = tf.placeholder(dtype=tf.float32, shape=[None, state_size])
         
        # Neural net starts here
         
        hidden_layer = tf.layers.dense(self.input_layer, 8, activation=tf.nn.relu, kernel_initializer=initializer)
        hidden_layer_2 = tf.layers.dense(hidden_layer, 8, activation=tf.nn.relu, kernel_initializer=initializer)
         
        # Output of neural net
        out = tf.layers.dense(hidden_layer_2, num_actions, activation=None)
         
        self.outputs = tf.nn.softmax(out)
        self.choice = tf.argmax(self.outputs, axis=1)
         
        # Training Procedure
        self.rewards = tf.placeholder(shape=[None, ], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[None, ], dtype=tf.int32)
         
        one_hot_actions = tf.one_hot(self.actions, num_actions)
         
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=one_hot_actions)
         
        self.loss = tf.reduce_mean(cross_entropy * self.rewards)
         
        self.gradients = tf.gradients(self.loss, tf.trainable_variables())
         
        # Create a placeholder list for gradients
        self.gradients_to_apply = []
        for index, variable in enumerate(tf.trainable_variables()):
            gradient_placeholder = tf.placeholder(tf.float32)
            self.gradients_to_apply.append(gradient_placeholder)
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        self.update_gradients = optimizer.apply_gradients(zip(self.gradients_to_apply, tf.trainable_variables()))

Discounting and Normalizing Rewards
---
In order to determine how "successful" a given action is, the policy gradient algorithm evaluates each action based on how many rewards were earned after it was performed in an episode.

The discount rewards function goes through each time step of an episode and tracks the total rewards earned from each step to the end of the episode.

For example, if an episode took 10 steps to finish, and the agent earns 1 point of reward every step, the rewards for each frame would be stored as 
`[10, 9, 8, 7, 6, 5, 4, 3, 2, 1]`

This allows the agent to credit early actions that didn't lose the game with future success, and later actions (that likely resulted in the end of the game) to get less credit.

One disadvantage of arranging rewards like this is that early actions didn't necessarily directly contribute to later rewards, so a **discount factor** is applied that scales rewards down over time. A discount factor < 1 means that rewards earned closer to the current time step will be worth more than rewards earned later.

With our reward example above, if we applied a discount factor of .90, the rewards would be stored as
`[ 6.5132156   6.12579511  5.6953279   5.217031    4.68559     4.0951      3.439
  2.71        1.9         1. ]`

This means that the early actions still get more credit than later actions, but not the full value of the rewards for the entire episode.

Finally, the rewards are normalized to lower the variance between reward values in longer or shorter episodes.

You can tweak the discount factor as one of the hyperparameters of your model to find one that fits your task the best!

In [6]:
# TODO Create the discounted and normalized rewards function
discount_rate = 0.95
 
def discount_normalize_rewards(rewards):
    discounted_rewards = np.zeros_like(rewards)
    total_rewards = 0
     
    for i in reversed(range(len(rewards))):
        total_rewards = total_rewards * discount_rate + rewards[i]
        discounted_rewards[i] = total_rewards
     
    discounted_rewards -= np.mean(discounted_rewards)
    discounted_rewards /= np.std(discounted_rewards)
     
    return discounted_rewards

Training Procedure
---
The agent will play games and record the history of the episode. At the end of every game, the episode's history will be processed to calculate the **gradients** that the model learned from that episode.

Every few games the calculated gradients will be applied, updating the model's parameters with the lessons from the games so far.

While training, you'll keep track of average scores and render the environment occasionally to see your model's progress.

In [7]:
# TODO Create the training loop
tf.reset_default_graph()
 
# Modify these to match shape of actions and states in your environment
num_actions = 2
state_size = 4
 
path = "./cartpole-pg/"
 
training_episodes = 1000
max_steps_per_episode = 10000
episode_batch_size = 5
 
agent = Agent(num_actions, state_size)
 
init = tf.global_variables_initializer()
 
saver = tf.train.Saver(max_to_keep=2)
 
if not os.path.exists(path):
    os.makedirs(path)
 
with tf.Session() as sess:
    sess.run(init)
     
    total_episode_rewards = []
     
    # Create a buffer of 0'd gradients
    gradient_buffer = sess.run(tf.trainable_variables())
    for index, gradient in enumerate(gradient_buffer):
        gradient_buffer[index] = gradient * 0
 
    for episode in range(training_episodes):
 
        state = env.reset()
         
        episode_history = []
        episode_rewards = 0
         
        for step in range(max_steps_per_episode):
             
            if episode % 100 == 0:
                env.render()
             
            # Get weights for each action
            action_probabilities = sess.run(agent.outputs, feed_dict={agent.input_layer: [state]})
            print(action_probabilities)
            action_choice = np.random.choice(range(num_actions), p=action_probabilities[0])
            print(action_choice)
             
            state_next, reward, done, _ = env.step(action_choice)
            episode_history.append([state, action_choice, reward, state_next])
            state = state_next
             
            episode_rewards += reward
             
            if done or step + 1 == max_steps_per_episode:
                total_episode_rewards.append(episode_rewards)
                episode_history = np.array(episode_history)
                episode_history[:,2] = discount_normalize_rewards(episode_history[:,2])
                 
                ep_gradients = sess.run(agent.gradients, feed_dict={agent.input_layer: np.vstack(episode_history[:, 0]),
                                                                    agent.actions: episode_history[:, 1],
                                                                    agent.rewards: episode_history[:, 2]})
                # add the gradients to the grad buffer:
                for index, gradient in enumerate(ep_gradients):
                    gradient_buffer[index] += gradient
                 
                break
             
        if episode % episode_batch_size == 0:
         
            feed_dict_gradients = dict(zip(agent.gradients_to_apply, gradient_buffer))
             
            sess.run(agent.update_gradients, feed_dict=feed_dict_gradients)
             
            for index, gradient in enumerate(gradient_buffer):
                gradient_buffer[index] = gradient * 0
                 
        if episode % 100 == 0:
            saver.save(sess, path + "pg-checkpoint", episode)
            print("Average reward / 100 eps: " + str(np.mean(total_episode_rewards[-100:])))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



[2018-08-03 10:34:05,425] From <ipython-input-5-09f7aad063ea>:26: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



[[0.49723607 0.50276387]]
1
[[0.43594557 0.5640545 ]]
1
[[0.36091578 0.6390842 ]]
1
[[0.2903715 0.7096285]]
1
[[0.22695674 0.7730433 ]]
0
[[0.27868316 0.7213168 ]]
1
[[0.21550758 0.78449243]]
0
[[0.26359284 0.7364071 ]]
1
[[0.20140174 0.7985983 ]]
1
[[0.14944082 0.85055923]]
1
[[0.10785232 0.8921477 ]]
1
[[0.07584309 0.9241569 ]]
0
Average reward / 100 eps: 12.0
[[0.47091788 0.52908206]]
0
[[0.48536298 0.514637  ]]
1
[[0.46441352 0.5355865 ]]
1
[[0.3702201  0.62977993]]
0
[[0.4583744  0.54162556]]
0
[[0.48674914 0.5132509 ]]
1
[[0.45092848 0.5490716 ]]
0
[[0.48756766 0.51243234]]
0
[[0.4812864 0.5187136]]
0
[[0.47483507 0.5251649 ]]
0
[[0.4679743  0.53202564]]
1
[[0.47280738 0.52719265]]
1
[[0.47820342 0.52179664]]
1
[[0.48468453 0.51531553]]
0
[[0.47673556 0.5232644 ]]
1
[[0.4830347 0.5169653]]
0
[[0.47515333 0.5248466 ]]
0
[[0.4670535 0.5329465]]
1
[[0.47223702 0.527763  ]]
1
[[0.4783484  0.52165157]]
0
[[0.47028974 0.5297103 ]]
1
[[0.47730494 0.52269506]]
1
[[0.49580044 0.5041996 ]]

[[0.03284866 0.9671513 ]]
1
[[0.01943025 0.9805697 ]]
1
[[0.01120893 0.9887911 ]]
1
[[0.00631024 0.9936898 ]]
1
[[0.00346793 0.9965321 ]]
1
[[0.41940376 0.58059615]]
0
[[0.4419806  0.55801946]]
0
[[0.43112513 0.56887484]]
0
[[0.42018726 0.57981277]]
0
[[0.4090789  0.59092104]]
1
[[0.41938072 0.5806192 ]]
1
[[0.4293563 0.5706437]]
1
[[0.43908727 0.5609127 ]]
0
[[0.42670035 0.5732996 ]]
0
[[0.41424945 0.5857506 ]]
1
[[0.4232729  0.57672703]]
1
[[0.43024236 0.56975764]]
0
[[0.41764775 0.5823523 ]]
1
[[0.42375553 0.5762444 ]]
1
[[0.42965585 0.57034415]]
1
[[0.41005656 0.58994347]]
0
[[0.42234442 0.57765555]]
1
[[0.41858238 0.5814176 ]]
0
[[0.4146204 0.5853796]]
1
[[0.4174714 0.5825286]]
0
[[0.40631336 0.59368664]]
0
[[0.4333829  0.56661713]]
1
[[0.32096565 0.6790343 ]]
0
[[0.43347886 0.5665211 ]]
0
[[0.4396081 0.5603919]]
0
[[0.42853418 0.5714658 ]]
0
[[0.4173704  0.58262956]]
0
[[0.40603063 0.59396935]]
1
[[0.41602597 0.5839741 ]]
1
[[0.42566767 0.57433236]]
0
[[0.41325468 0.58674526]]
0


1
[[0.06604125 0.9339587 ]]
1
[[0.03797175 0.9620282 ]]
1
[[0.02124174 0.9787582 ]]
1
[[0.01159745 0.98840255]]
1
[[0.0061896 0.9938105]]
1
[[0.0032316 0.9967685]]
1
[[0.36012694 0.6398731 ]]
1
[[0.2649748  0.73502517]]
1
[[0.1775122 0.8224879]]
1
[[0.11145765 0.8885424 ]]
1
[[0.06725242 0.9327476 ]]
1
[[0.03928963 0.9607104 ]]
1
[[0.02233319 0.9776668 ]]
1
[[0.01238613 0.9876138 ]]
1
[[0.00671153 0.99328846]]
0
[[0.01044038 0.98955965]]
1
[[0.00549161 0.99450845]]
1
[[0.36025107 0.63974893]]
1
[[0.25538662 0.74461347]]
0
[[0.35849592 0.641504  ]]
1
[[0.25308716 0.74691284]]
1
[[0.16289182 0.83710814]]
1
[[0.09957064 0.90042937]]
1
[[0.0585165  0.94148356]]
1
[[0.03332868 0.9666713 ]]
1
[[0.0184899  0.98151016]]
1
[[0.01001911 0.9899809 ]]
1
[[0.00530993 0.99469006]]
1
[[0.3430465 0.6569535]]
0
[[0.37775406 0.6222459 ]]
1
[[0.34630477 0.6536952 ]]
1
[[0.24785104 0.752149  ]]
1
[[0.16180986 0.83819014]]
1
[[0.09866218 0.90133786]]
1
[[0.05779878 0.94220126]]
1
[[0.03279128 0.96720874]]


...

1
[[0.00113787 0.99886215]]
1
[[4.5994602e-04 9.9954009e-01]]
1
[[1.8167014e-04 9.9981838e-01]]
1
[[6.991656e-05 9.999300e-01]]
1
[[2.6149963e-05 9.9997389e-01]]
1
[[9.4850384e-06 9.9999046e-01]]
1
[[3.3316817e-06 9.9999666e-01]]
1
[[0.01249924 0.9875008 ]]
1
[[0.00653488 0.9934651 ]]
1
[[0.00340498 0.99659497]]
1
[[0.00170186 0.9982981 ]]
1
[[6.624495e-04 9.993375e-01]]
1
[[2.5249925e-04 9.9974746e-01]]
1
[[9.3991424e-05 9.9990594e-01]]
1
[[3.4086104e-05 9.9996591e-01]]
1
[[1.2019525e-05 9.9998796e-01]]
1
[[0.01450716 0.9854929 ]]
1
[[0.00767765 0.9923224 ]]
1
[[0.00405032 0.9959496 ]]
1
[[0.00212826 0.99787176]]
1
[[9.3208614e-04 9.9906796e-01]]
1
[[3.644476e-04 9.996356e-01]]
1
[[1.3925631e-04 9.9986076e-01]]
1
[[5.1859210e-05 9.9994814e-01]]
1
[[1.8778446e-05 9.9998116e-01]]
1
[[6.6003904e-06 9.9999344e-01]]
1
[[0.01365967 0.9863403 ]]
1
[[0.00739851 0.99260145]]
1
[[0.00397363 0.9960264 ]]
1
[[0.00212561 0.9978744 ]]
1
[[0.00100139 0.9989986 ]]
1
[[4.0343386e-04 9.9959654e-01]]
1


[[5.9298898e-05 9.9994075e-01]]
1
[[2.0328549e-05 9.9997962e-01]]
1
[[6.7664173e-06 9.9999321e-01]]
1
[[0.00968691 0.99031305]]
1
[[0.0048707  0.99512935]]
1
[[0.002442 0.997558]]
1
[[0.00121924 0.9987808 ]]
1
[[5.1520375e-04 9.9948478e-01]]
1
[[1.9022732e-04 9.9980980e-01]]
1
[[6.8504087e-05 9.9993145e-01]]
1
[[2.3992876e-05 9.9997604e-01]]
1
[[8.153615e-06 9.999919e-01]]
1
[[0.00870868 0.9912913 ]]
1
[[0.00438819 0.9956118 ]]
1
[[0.00220498 0.997795  ]]
1
[[0.0011033 0.9988967]]
1
[[4.6546926e-04 9.9953461e-01]]
1
[[1.7234693e-04 9.9982762e-01]]
1
[[6.222768e-05 9.999378e-01]]
1
[[2.1846956e-05 9.9997818e-01]]
1
[[7.440358e-06 9.999926e-01]]
1
[[2.4539372e-06 9.9999750e-01]]
1
[[0.00953612 0.99046385]]
1
[[0.00475339 0.9952466 ]]
1
[[0.00236259 0.9976374 ]]
1
[[0.00116939 0.9988306 ]]
1
[[5.168180e-04 9.994831e-01]]
1
[[1.8908014e-04 9.9981099e-01]]
1
[[6.745419e-05 9.999325e-01]]
1
[[2.3398503e-05 9.9997663e-01]]
1
[[7.8732082e-06 9.9999213e-01]]
1
[[0.00940355 0.9905964 ]]
1
[[0.00

1
[[0.00321319 0.99678683]]
1
[[0.00154691 0.998453  ]]
1
[[7.4159278e-04 9.9925834e-01]]
1
[[3.418721e-04 9.996581e-01]]
1
[[1.21740355e-04 9.99878287e-01]]
1
[[4.2226187e-05 9.9995780e-01]]
1
[[1.4222173e-05 9.9998581e-01]]
1
[[4.6394307e-06 9.9999535e-01]]
1
[[1.4630622e-06 9.9999857e-01]]
1
[[0.00766535 0.99233466]]
1
[[0.00366235 0.99633765]]
1
[[0.00174531 0.9982547 ]]
1
[[8.2830724e-04 9.9917173e-01]]
1
[[3.678677e-04 9.996321e-01]]
1
[[1.2897601e-04 9.9987102e-01]]
1
[[4.4055512e-05 9.9995589e-01]]
1
[[1.4617785e-05 9.9998534e-01]]
1
[[4.7000672e-06 9.9999535e-01]]
1
[[0.00802276 0.9919773 ]]
1
[[0.00375405 0.9962459 ]]
1
[[0.00174713 0.9982528 ]]
1
[[8.1038463e-04 9.9918956e-01]]
1
[[3.4737328e-04 9.9965262e-01]]
1
[[1.1872682e-04 9.9988127e-01]]
1
[[3.957634e-05 9.999604e-01]]
1
[[1.282926e-05 9.999871e-01]]
1
[[4.0349669e-06 9.9999595e-01]]
1
[[0.006909 0.993091]]
1
[[0.00331071 0.99668926]]
1
[[0.00158237 0.9984176 ]]
1
[[7.5306540e-04 9.9924695e-01]]
1
[[3.5400494e-04 9.99

[[0.00313072 0.99686927]]
1
[[0.00143822 0.9985618 ]]
1
[[6.5826031e-04 9.9934167e-01]]
1
[[2.9949241e-04 9.9970055e-01]]
1
[[1.1078736e-04 9.9988925e-01]]
1
[[3.7282472e-05 9.9996269e-01]]
1
[[1.2186887e-05 9.9998784e-01]]
1
[[3.858784e-06 9.999962e-01]]
1
[[1.1810821e-06 9.9999881e-01]]
1
[[0.00708835 0.99291164]]
1
[[0.00318076 0.99681926]]
1
[[0.00142375 0.9985763 ]]
1
[[6.3471345e-04 9.9936527e-01]]
1
[[2.6515924e-04 9.9973482e-01]]
1
[[8.759483e-05 9.999124e-01]]
1
[[2.8189310e-05 9.9997187e-01]]
1
[[8.812607e-06 9.999912e-01]]
1
[[0.00750632 0.9924936 ]]
1
[[0.00340572 0.99659425]]
1
[[0.00154178 0.99845827]]
1
[[6.9534546e-04 9.9930465e-01]]
1
[[3.0897016e-04 9.9969101e-01]]
1
[[1.0381935e-04 9.9989617e-01]]
1
[[3.3998578e-05 9.9996603e-01]]
1
[[1.0818983e-05 9.9998915e-01]]
1
[[3.3375586e-06 9.9999666e-01]]
1
[[0.00670393 0.9932961 ]]
1
[[0.00309422 0.99690574]]
1
[[0.00142512 0.99857485]]
1
[[6.538352e-04 9.993462e-01]]
1
[[2.9814825e-04 9.9970180e-01]]
1
[[1.1000320e-04 9.99

[[5.424412e-06 9.999945e-01]]
1
[[1.6860740e-06 9.9999833e-01]]
1
[[0.00740811 0.99259186]]
1
[[0.00345106 0.99654895]]
1
[[0.00160368 0.99839634]]
1
[[7.4217503e-04 9.9925786e-01]]
1
[[3.2633622e-04 9.9967372e-01]]
1
[[1.1285601e-04 9.9988711e-01]]
1
[[3.8023652e-05 9.9996197e-01]]
1
[[1.2443876e-05 9.9998760e-01]]
1
[[3.9461652e-06 9.9999607e-01]]
1
[[0.00777931 0.9922207 ]]
1
[[0.00365056 0.99634945]]
1
[[0.0017089 0.9982911]]
1
[[7.9674594e-04 9.9920326e-01]]
1
[[3.6919152e-04 9.9963081e-01]]
1
[[1.3726769e-04 9.9986267e-01]]
1
[[4.6825484e-05 9.9995315e-01]]
1
[[1.5515285e-05 9.9998450e-01]]
1
[[4.980443e-06 9.999950e-01]]
1
[[1.5459308e-06 9.9999845e-01]]
1
[[0.00652597 0.99347407]]
1
[[0.00310923 0.99689084]]
1
[[0.00147758 0.99852246]]
1
[[6.9910591e-04 9.9930084e-01]]
1
[[3.2859994e-04 9.9967146e-01]]
1
[[1.2515335e-04 9.9987483e-01]]
1
[[4.356836e-05 9.999565e-01]]
1
[[1.4722644e-05 9.9998522e-01]]
1
[[4.8160250e-06 9.9999523e-01]]
1
[[1.5218312e-06 9.9999845e-01]]
1
[[0.0083

Testing the Model
---

This cell will run through games choosing actions without the learning process so you can see how your model has learned!

In [10]:
# TODO Create the testing loop
testing_episodes = 5
 
with tf.Session() as sess:
    checkpoint = tf.train.get_checkpoint_state(path)
    saver.restore(sess,checkpoint.model_checkpoint_path)
 
    for episode in range(testing_episodes):
     
            state = env.reset()
     
            episode_rewards = 0
             
            for step in range(max_steps_per_episode):
                 
                env.render()
                 
                # Get Action
                action_argmax = sess.run(agent.choice, feed_dict={agent.input_layer: [state]})
                action_choice = action_argmax[0]
                 
                state_next, reward, done, _ = env.step(action_choice)
                state = state_next
                 
                episode_rewards += reward
                 
                if done or step + 1 == max_steps_per_episode:
                   print("Rewards for episode " + str(episode) + ": " + str(episode_rewards))
                   break

INFO:tensorflow:Restoring parameters from ./cartpole-pg/pg-checkpoint-900


[2018-08-03 00:06:08,622] Restoring parameters from ./cartpole-pg/pg-checkpoint-900


Rewards for episode 0: 251.0
Rewards for episode 1: 291.0
Rewards for episode 2: 306.0
Rewards for episode 3: 264.0
Rewards for episode 4: 268.0


In [11]:
# Run to close the environment
env.close()